In [1]:
from pathlib import Path

from mndot_bid_etl.transform.abstract import read_abstract_csv
from mndot_bid_etl.transform.transform_bid import *

In [2]:
csv_file = Path("../data/csv/2022/220002.csv").resolve()
abstract = read_abstract_csv(csv_file)
bid_df = abstract.bid_df

In [3]:
search_strings = [
    "ContractId",
    "SectionDescription",
    "LineNumber",
    "ItemDescription",
    "UnitPrice",
    "UnitName",
    "Ext",
]
bid_df_dropped = drop_bid_columns(bid_df, search_strings)
bid_df_dropped.head()

,ItemNumber,Quantity,Engineers (Unit Price),0000198793 (Unit Price),0000210000 (Unit Price),0000207897 (Unit Price)
0,2011601/01000,1,15000,"$9,800.00","$12,000.00","$9,800.00"
1,2021501/00010,1,250000,"$209,000.00","$317,000.00","$216,000.00"
2,2051501/00010,1,1000,$1.00,$1.00,$1.00
3,2101505/00020,1.4,8000,"$3,500.00","$3,500.00","$8,000.00"
4,2101505/00030,1.4,4000,"$3,700.00","$3,700.00","$5,500.00"


In [4]:
bid_df_renamed = rename_bid_columns(bid_df_dropped)
bid_df_renamed.head()

,item_id,quantity,engineers,0000198793,0000210000,0000207897
0,2011601/01000,1,15000,"$9,800.00","$12,000.00","$9,800.00"
1,2021501/00010,1,250000,"$209,000.00","$317,000.00","$216,000.00"
2,2051501/00010,1,1000,$1.00,$1.00,$1.00
3,2101505/00020,1.4,8000,"$3,500.00","$3,500.00","$8,000.00"
4,2101505/00030,1.4,4000,"$3,700.00","$3,700.00","$5,500.00"
